In [1]:
import pandas as pd

In [2]:
from glob import glob 

In [3]:
print("Bem Vindo(a) :D ")

Bem Vindo(a) :D 


In [4]:
print("Lendo bases...")

Lendo bases...


### Leitura das bases

In [5]:
df_base_atestados = pd.read_excel(glob("bases/base_ambulatorio/*")[0], index_col = None)

In [6]:
df_base_lojas_centro = pd.read_excel(glob("bases/base_nomes_lojas/*")[0], index_col = None)

In [7]:
df_base_lojas_copiado = df_base_lojas_centro.copy()

In [8]:
df_base_lojas_genero = df_base_lojas_centro.copy()

In [9]:
df_base_atestados.columns = ['data_sap','matricula','nome','idade','sexo','funcao','loja','centro','data_inicial','data_final','vai_inss','dias_perdidos','cib_10','grupo_doenca','lcdo_soc','','','']

In [10]:
df_base_atestados_filtrado = df_base_atestados[['data_sap','matricula','nome','idade','sexo','funcao','loja','centro','data_inicial','data_final','vai_inss','dias_perdidos','cib_10','grupo_doenca','lcdo_soc']].copy()

In [11]:
df_base_atestados_filtrado = df_base_atestados_filtrado[~df_base_atestados_filtrado['matricula'].isnull()].copy()

In [12]:
df_base_atestados_filtrado = df_base_atestados_filtrado[df_base_atestados_filtrado['nome'] != 'NOME'].copy()

#### filtro das colunas importantes

In [13]:
df_atestados_meses = df_base_atestados_filtrado[['data_inicial','dias_perdidos','loja','centro','sexo','idade','funcao','grupo_doenca']].copy()

### Afirmando que os nomes das lojas estarão em caixa alta

In [14]:
df_atestados_meses['loja'] = df_atestados_meses['loja'].astype(str)

In [15]:
df_atestados_meses['loja'] = df_atestados_meses['loja'].str.upper() 

In [16]:
### Pegando somente o dado que necessita que é o mês
df_atestados_meses['data_inicial'] = df_atestados_meses['data_inicial'].astype(str)

In [17]:
df_atestados_meses['data_inicial'] = df_atestados_meses['data_inicial'].str.replace(".","-")
df_atestados_meses['data_inicial'] = df_atestados_meses['data_inicial'].str.replace(":","-")
df_atestados_meses['data_inicial'] = df_atestados_meses['data_inicial'].str.replace(" ","-")

In [18]:
df_atestados_meses['mes'] = df_atestados_meses['data_inicial'].apply(lambda item: str(item).split('-')[1])

#  SEPARAÇÃO DOS DATA FRAMES POR FILTRO ESPECÍFICO DE CADA SITUAÇÃO 


## ------ Filtro para Gerar Quantidade de licenças médicas por Unidade

In [19]:
df_licencas_unidade = df_atestados_meses[['mes','centro','dias_perdidos']].copy()

In [20]:
df_faixa_etaria = df_atestados_meses[['mes','idade']].copy()

In [21]:
df_licencas_genero = df_atestados_meses[['mes','centro','sexo']].copy()

In [22]:
df_licencas_genero.head()

,mes,centro,sexo
0,05,107.0,F
1,05,114.0,F
2,05,201.0,F
3,05,201.0,F
4,05,202.0,F


In [23]:
df_dias_perdidos_unidade = df_licencas_unidade.copy()

In [24]:
df_atestados_meses.drop(columns = {'data_inicial'}, inplace = True)

In [25]:
df_atestados_meses['mes'] = df_atestados_meses['mes'].astype(int)

In [26]:
df_atestados_meses['mes_copiado'] = df_atestados_meses['mes']

## QUANTIDADE DE DIAS PERDIDOS E LICENÇAS POR MÊS

In [27]:
df_atestados_meses = df_atestados_meses.groupby('mes', as_index = False).agg({'mes_copiado': 'count','dias_perdidos':'sum'})

In [28]:
df_atestados_meses


,mes,mes_copiado,dias_perdidos
0,5,151,355
1,6,162,341
2,7,200,485
3,8,199,426
4,9,153,332
5,10,199,411


In [29]:
df_atestados_meses.rename(columns = {'mes_copiado':'QTE DE LICENÇAS MÉDICAS','dias_perdidos':'QTE DIAS PERDIDOS'}, inplace = True)

In [30]:
nome_mes = {1: 'JAN', 2: 'FEV', 3:'MARC', 4:'ABR', 5:'MAIO', 6:'JUN', 7:'JUL', 8:'AGO', 9:'SET', 10:'OUT', 11:'NOV' , 12: 'DEZ'}

In [31]:
df_atestados_meses['nome_mes'] = df_atestados_meses['mes'].map(nome_mes)

In [32]:
df_atestados_meses

,mes,QTE DE LICENÇAS MÉDICAS,QTE DIAS PERDIDOS,nome_mes
0,5,151,355,MAIO
1,6,162,341,JUN
2,7,200,485,JUL
3,8,199,426,AGO
4,9,153,332,SET
5,10,199,411,OUT


In [33]:
df_atestados_meses.sort_values(by = ['mes'], ascending = True, inplace = True)

In [34]:
df_atestados_meses

,mes,QTE DE LICENÇAS MÉDICAS,QTE DIAS PERDIDOS,nome_mes
0,5,151,355,MAIO
1,6,162,341,JUN
2,7,200,485,JUL
3,8,199,426,AGO
4,9,153,332,SET
5,10,199,411,OUT


In [35]:
df_atestados_meses = df_atestados_meses.T 

In [36]:
df_atestados_meses

,0,1,2,3,4,5
mes,5,6,7,8,9,10
QTE DE LICENÇAS MÉDICAS,151,162,200,199,153,199
QTE DIAS PERDIDOS,355,341,485,426,332,411
nome_mes,MAIO,JUN,JUL,AGO,SET,OUT


In [37]:
#Reorganizando os meses com os respectivos nomes
df_atestados_meses.columns = df_atestados_meses.loc['nome_mes'] 
df_atestados_meses.columns.name = None

In [38]:
df_atestados_meses

,MAIO,JUN,JUL,AGO,SET,OUT
mes,5,6,7,8,9,10
QTE DE LICENÇAS MÉDICAS,151,162,200,199,153,199
QTE DIAS PERDIDOS,355,341,485,426,332,411
nome_mes,MAIO,JUN,JUL,AGO,SET,OUT


In [39]:
df_atestados_meses.drop(['mes', 'nome_mes'], inplace = True)

In [40]:
df_atestados_meses

,MAIO,JUN,JUL,AGO,SET,OUT
QTE DE LICENÇAS MÉDICAS,151,162,200,199,153,199
QTE DIAS PERDIDOS,355,341,485,426,332,411


In [41]:
lista_colunas = df_atestados_meses.columns

lista_meses = ["JAN", "FEV", "MAR", "ABR", "MAIO", "JUN","JUL","AGO","SET","OUT","NOV","DEZ"]

lista_meses = [x for x in lista_meses if x not in lista_colunas] 

In [42]:
lista_meses

['JAN', 'FEV', 'MAR', 'ABR', 'NOV', 'DEZ']

In [43]:
for mes in lista_meses:
    df_atestados_meses[mes] = 0

In [44]:
df_atestados_meses.rename(columns = {'JAN': 1,"FEV":2,"MAR":3,"ABR":4,"MAIO":5,"JUN":6,"JUL":7,"AGO":8,"SET":9,"OUT":10,"NOV":11,"DEZ":12}, inplace = True)

In [45]:
df_atestados_meses.sort_index(axis = 1, inplace = True)

In [46]:
df_atestados_meses.rename(columns = {1: 'JANEIRO',2:"FEV",3:"MAR",4:"ABR",5:"MAIO",6:"JUN",7:"JUL",8:"AGO",9:"SET",10:"OUT",11:"NOV",12:"DEZ"}, inplace = True)

In [47]:
df_atestados_meses

,JANEIRO,FEV,MAR,ABR,MAIO,JUN,JUL,AGO,SET,OUT,NOV,DEZ
QTE DE LICENÇAS MÉDICAS,0,0,0,0,151,162,200,199,153,199,0,0
QTE DIAS PERDIDOS,0,0,0,0,355,341,485,426,332,411,0,0


## QUANTIDADE DE LICENÇAS MÉDICAS POR UNIDADE

### Função que tem por finalidade dizer onde tem traço e trazer so a segunda posição, ou seja, restando somente o nome da loja

In [48]:
# def consertando(row):
#     if '-' in row['loja']:
#         row['loja'] = row['loja'].split('-')[1]
#     return row

# df_licencas_unidade = df_licencas_unidade.apply(consertando, axis = 1)

## Contando pelo centro a quantidade de licenças médicas por unidade

## data frame para renomear as lojas

In [49]:
df_base_lojas_centro.shape

(43, 2)

In [50]:
df_base_lojas_centro.head()

,loja,centro
0,ARIQUEMES,206
1,AVENIDA,103
2,BARROSO,107
3,BEMOL ONLINE,102
4,CAMAPUÃ,118


In [51]:
df_licencas_unidade['centro'] = df_licencas_unidade['centro'].astype(int)

In [52]:
df_licencas_unidade['mes_unidades_copiado'] = df_licencas_unidade['mes'].copy()

In [53]:
df_licencas_unidade['mes'] = df_licencas_unidade['mes'].astype(int)

In [54]:
df_licencas_unidade.head()

,mes,centro,dias_perdidos,mes_unidades_copiado
0,5,107,1,05
1,5,114,1,05
2,5,201,7,05
3,5,201,2,05
4,5,202,2,05


In [55]:
lista_ordenacao = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j','k','l']

In [56]:
for mes in df_licencas_unidade['mes'].unique():
    
    df_aux = df_licencas_unidade[df_licencas_unidade['mes'] == mes].copy()
   
    df_agrupado = df_aux.groupby(['centro'], as_index = False).agg({'mes_unidades_copiado': 'count'}).copy()
       
    df_agrupado.rename(columns={"mes_unidades_copiado": lista_ordenacao[mes]}, inplace = True)
    
    df_base_lojas_centro = pd.merge(df_base_lojas_centro, df_agrupado, on = ['centro'], how = 'outer')
    
    df_base_lojas_centro.fillna(0, inplace = True)


In [57]:
df_base_lojas_centro.rename(columns = {"centro":"a_centro", "loja":'a_loja'}, inplace = True)

In [58]:
df_base_lojas_centro.head()

,a_loja,a_centro,f,g,h,i,j,k
0,ARIQUEMES,206,0.0,0.0,0.0,0.0,0.0,0.0
1,AVENIDA,103,6.0,8.0,9.0,15.0,5.0,15.0
2,BARROSO,107,11.0,4.0,9.0,2.0,2.0,4.0
3,BEMOL ONLINE,102,0.0,4.0,3.0,3.0,1.0,2.0
4,CAMAPUÃ,118,8.0,10.0,10.0,19.0,18.0,13.0


In [59]:
df_base_lojas_centro.sort_index(axis = 1, inplace = True)

In [60]:
dic_mes_alfabeto = {'a': 'JAN', 'b': 'FEV', 'c': 'MAR', 'd': 'ABR', 'e':'MAIO', 'f':'JUN', 'g':'JUL', 'h':'AGO', 'i':'SET', 'j':'OUT', 'k':'NOV','l':'DEZ'}

In [61]:
df_base_lojas_centro.head()

,a_centro,a_loja,f,g,h,i,j,k
0,206,ARIQUEMES,0.0,0.0,0.0,0.0,0.0,0.0
1,103,AVENIDA,6.0,8.0,9.0,15.0,5.0,15.0
2,107,BARROSO,11.0,4.0,9.0,2.0,2.0,4.0
3,102,BEMOL ONLINE,0.0,4.0,3.0,3.0,1.0,2.0
4,118,CAMAPUÃ,8.0,10.0,10.0,19.0,18.0,13.0


In [62]:
for name in df_base_lojas_centro.columns[2:]:
    df_base_lojas_centro.rename(columns = {name: dic_mes_alfabeto[name]},inplace = True)

In [63]:
df_base_lojas_centro.head()

,a_centro,a_loja,JUN,JUL,AGO,SET,OUT,NOV
0,206,ARIQUEMES,0.0,0.0,0.0,0.0,0.0,0.0
1,103,AVENIDA,6.0,8.0,9.0,15.0,5.0,15.0
2,107,BARROSO,11.0,4.0,9.0,2.0,2.0,4.0
3,102,BEMOL ONLINE,0.0,4.0,3.0,3.0,1.0,2.0
4,118,CAMAPUÃ,8.0,10.0,10.0,19.0,18.0,13.0


In [64]:
df_base_lojas_centro.columns

Index(['a_centro', 'a_loja', 'JUN', 'JUL', 'AGO', 'SET', 'OUT', 'NOV'], dtype='object')

In [65]:
lista_colunas = df_base_lojas_centro.columns

lista_semestre = ["JAN", "FEV", "MAR", "ABR", "MAIO", "JUN","JUL","AGO","SET","OUT","NOV","DEZ"]

if "JUL" in lista_colunas:
    lista_meses = [x for x in lista_semestre if x not in lista_colunas]
else:
    lista_meses = [x for x in lista_semestre if x not in lista_colunas]

In [66]:
lista_meses

['JAN', 'FEV', 'MAR', 'ABR', 'MAIO', 'DEZ']

In [67]:
for mes in lista_meses:
    df_base_lojas_centro[mes] = 0

In [68]:
df_base_lojas_centro = df_base_lojas_centro[["a_loja","JAN", "FEV", "MAR", "ABR", "MAIO", "JUN","JUL","AGO","SET","OUT","NOV","DEZ"]].copy()

In [69]:
df_base_lojas_centro.rename(columns = {'a_loja':'LOJA'}, inplace = True)

In [70]:
df_qtd_atestados_unidade = df_base_lojas_centro.copy()

##  QUANTIDADE DE DIAS PERDIDOS POR MÊS

In [71]:
df_dias_perdidos_unidade['mes'] = df_dias_perdidos_unidade['mes'].astype(int)

In [72]:
df_dias_perdidos_unidade['centro'] = df_dias_perdidos_unidade['centro'].astype(int)

In [73]:
for mes in df_dias_perdidos_unidade['mes'].unique():
    
    df_aux = df_dias_perdidos_unidade[df_dias_perdidos_unidade['mes'] == mes].copy()
   
    df_agrupando = df_aux.groupby(['centro'], as_index = False).agg({'dias_perdidos': 'sum'}).copy()
       
    df_agrupando.rename(columns = {"dias_perdidos": lista_ordenacao[mes]}, inplace = True)
    
    df_base_lojas_copiado = pd.merge(df_base_lojas_copiado, df_agrupando, on = ['centro'], how = 'outer')
    
    df_base_lojas_copiado.fillna(0, inplace = True)

In [74]:
# df_base_lojas_copiado.tail()

In [75]:
df_base_lojas_copiado.rename(columns = {"centro":"a_centro", "loja":'a_loja'}, inplace = True)

In [76]:
df_base_lojas_copiado.sort_index(axis = 1, inplace = True)

In [77]:
for name in df_base_lojas_copiado.columns[2:]:
    df_base_lojas_copiado.rename(columns = {name: dic_mes_alfabeto[name]},inplace = True)

In [78]:
lista_colunas = df_base_lojas_copiado.columns

lista_semestre = ["JAN", "FEV", "MAR", "ABR", "MAIO", "JUN","JUL","AGO","SET","OUT","NOV","DEZ"]

if "JUL" in lista_colunas:
    lista_meses = [x for x in lista_semestre if x not in lista_colunas]
else:
    lista_meses = [x for x in lista_semestre if x not in lista_colunas]

In [79]:
for mes in lista_meses:
    df_base_lojas_copiado[mes] = 0

In [80]:
df_base_lojas_copiado = df_base_lojas_copiado[["a_loja","JAN", "FEV", "MAR", "ABR", "MAIO", "JUN","JUL","AGO","SET","OUT","NOV","DEZ"]].copy()

In [81]:
df_base_lojas_centro.rename(columns = {'a_loja':'LOJA'}, inplace = True)

In [82]:
df_qtd_dias_perdidos_unidade = df_base_lojas_centro.copy()

## QUANTITATIVO DE LICENÇAS MÉDICAS POR GÊNERO 

In [83]:
df_licencas_genero['centro'] = df_licencas_genero['centro'].astype(int)

In [84]:
df_licencas_genero['mes'] = df_licencas_genero['mes'].astype(int)

In [85]:
## deixandos todos em caixa alta
df_licencas_genero['sexo'] = df_licencas_genero['sexo'].str.upper()

In [86]:
df_licencas_genero.drop(columns = {"centro"}, inplace  = True)

In [87]:
df_feminino = df_licencas_genero[df_licencas_genero['sexo'] == 'F'].copy()

In [88]:
df_masculino = df_licencas_genero[df_licencas_genero['sexo'] == 'M'].copy()

In [89]:
df_feminino = df_feminino.groupby('mes', as_index = False).agg({'sexo': 'count'})

In [90]:
############################## CONTAGEM DOS FEMININOS #################

In [91]:
nome_mes_genero = {1: 'JAN', 2: 'FEV', 3:'MARC', 4:'ABR', 5:'MAIO', 6:'JUN', 7:'JUL', 8:'AGO', 9:'SET', 10:'OUT', 11:'NOV' , 12: 'DEZ'}

In [92]:
df_feminino['nome_mes_genero'] = df_feminino['mes'].map(nome_mes_genero)

In [93]:
df_feminino.rename(columns = {"sexo":"FEMININO"}, inplace = True)

In [94]:
df_feminino.sort_values(by = ['mes'], ascending = True, inplace = True)

In [95]:
df_feminino = df_feminino.T 

In [96]:
df_feminino

,0,1,2,3,4,5
mes,5,6,7,8,9,10
FEMININO,120,116,130,138,109,120
nome_mes_genero,MAIO,JUN,JUL,AGO,SET,OUT


In [97]:
#Reorganizando os meses com os respectivos nomes
df_feminino.columns = df_feminino.loc['nome_mes_genero'] 
df_feminino.columns.name = None

In [98]:
df_feminino

,MAIO,JUN,JUL,AGO,SET,OUT
mes,5,6,7,8,9,10
FEMININO,120,116,130,138,109,120
nome_mes_genero,MAIO,JUN,JUL,AGO,SET,OUT


In [99]:
df_feminino.drop(['mes', 'nome_mes_genero'], inplace = True)

In [100]:
df_feminino

,MAIO,JUN,JUL,AGO,SET,OUT
FEMININO,120,116,130,138,109,120


In [101]:
lista_colunas_genero = df_feminino.columns

lista_meses_genero = ["JAN", "FEV", "MAR", "ABR", "MAIO", "JUN","JUL","AGO","SET","OUT","NOV","DEZ"]

lista_meses_genero = [x for x in lista_meses_genero if x not in lista_colunas_genero] 

In [102]:
lista_meses_genero

['JAN', 'FEV', 'MAR', 'ABR', 'NOV', 'DEZ']

In [103]:
for mes in lista_meses_genero:
    df_feminino[mes] = 0

In [104]:
df_feminino

,MAIO,JUN,JUL,AGO,SET,OUT,JAN,FEV,MAR,ABR,NOV,DEZ
FEMININO,120,116,130,138,109,120,0,0,0,0,0,0


In [105]:
df_feminino.rename(columns = {'JAN': 1,"FEV":2,"MAR":3,"ABR":4,"MAIO":5,"JUN":6,"JUL":7,"AGO":8,"SET":9,"OUT":10,"NOV":11,"DEZ":12}, inplace = True)

In [106]:
df_feminino.sort_index(axis = 1, inplace = True)

In [107]:
df_feminino.rename(columns = {1: 'JANEIRO',2:"FEV",3:"MAR",4:"ABR",5:"MAIO",6:"JUN",7:"JUL",8:"AGO",9:"SET",10:"OUT",11:"NOV",12:"DEZ"}, inplace = True)

In [108]:
df_feminino

,JANEIRO,FEV,MAR,ABR,MAIO,JUN,JUL,AGO,SET,OUT,NOV,DEZ
FEMININO,0,0,0,0,120,116,130,138,109,120,0,0


In [109]:
#################### Inicio da contagem dos MASCULINOS ##############################

In [110]:
df_masculino = df_masculino.groupby('mes', as_index = False).agg({'sexo': 'count'})

In [111]:
df_masculino['nome_mes_genero'] = df_masculino['mes'].map(nome_mes_genero)

In [112]:
df_masculino.rename(columns = {"sexo":"MASCULINO"}, inplace = True)

In [113]:
df_masculino.sort_values(by = ['mes'], ascending = True, inplace = True)

In [114]:
df_masculino = df_masculino.T 

In [115]:
#Reorganizando os meses com os respectivos nomes
df_masculino.columns = df_masculino.loc['nome_mes_genero'] 
df_masculino.columns.name = None

In [116]:
df_masculino.drop(['mes', 'nome_mes_genero'], inplace = True)

In [117]:
lista_colunas_genero = df_masculino.columns

lista_meses_genero = ["JAN", "FEV", "MAR", "ABR", "MAIO", "JUN","JUL","AGO","SET","OUT","NOV","DEZ"]

lista_meses_genero = [x for x in lista_meses_genero if x not in lista_colunas_genero] 

In [118]:
for mes in lista_meses_genero:
    df_masculino[mes] = 0

In [119]:
df_masculino.rename(columns = {'JAN': 1,"FEV":2,"MAR":3,"ABR":4,"MAIO":5,"JUN":6,"JUL":7,"AGO":8,"SET":9,"OUT":10,"NOV":11,"DEZ":12}, inplace = True)

In [120]:
df_masculino.sort_index(axis = 1, inplace = True)

In [121]:
df_masculino.rename(columns = {1: 'JANEIRO',2:"FEV",3:"MAR",4:"ABR",5:"MAIO",6:"JUN",7:"JUL",8:"AGO",9:"SET",10:"OUT",11:"NOV",12:"DEZ"}, inplace = True)

In [122]:
df_final_genero = pd.concat([df_feminino,df_masculino])

In [123]:
df_final_genero

,JANEIRO,FEV,MAR,ABR,MAIO,JUN,JUL,AGO,SET,OUT,NOV,DEZ
FEMININO,0,0,0,0,120,116,130,138,109,120,0,0
MASCULINO,0,0,0,0,31,46,70,61,44,79,0,0


## Quantidade de Licenças médicas por intervalo de idade

In [124]:
df_faixa_etaria.head()

,mes,idade
0,05,55
1,05,31
2,05,30
3,05,30
4,05,27


In [125]:
df_faixa_etaria['idade'] = df_faixa_etaria['idade'].astype(int)
df_faixa_etaria['mes'] = df_faixa_etaria['mes'].astype(int)

In [126]:
# def get_faixa_etaria(idade):
#     if i

In [127]:
df_faixa_etaria.head()

,mes,idade
0,5,55
1,5,31
2,5,30
3,5,30
4,5,27


In [128]:
for mes in df_faixa_etaria['mes'].unique(): 
    
    df_aux = df_faixa_etaria[df_faixa_etaria['mes'] == mes].copy()
    
    df_16_29 = df_faixa_etaria[(df_faixa_etaria['idade']>=16) & (df_faixa_etaria['idade']<=29)]
    df_16_29 = df_16_29.groupby(['mes'], as_index = False).agg({'idade': 'count'})
    
    df_30_39 = df_faixa_etaria[(df_faixa_etaria['idade']>=30) & (df_faixa_etaria['idade']<=39)]
    df_30_39 = df_30_39.groupby(['mes'], as_index = False).agg({'idade': 'count'})
    
    df_40_49 = df_faixa_etaria[(df_faixa_etaria['idade']>=40) & (df_faixa_etaria['idade']<=49)]
    df_40_49 = df_40_49.groupby(['mes'], as_index = False).agg({'idade': 'count'})
        
    df_50_59 = df_faixa_etaria[(df_faixa_etaria['idade']>=50) & (df_faixa_etaria['idade']<=59)]
    df_50_59 = df_50_59.groupby(['mes'], as_index = False).agg({'idade': 'count'})
    
    df_acima_60 = df_faixa_etaria[(df_faixa_etaria['idade']>=60)]
    df_acima_60 = df_acima_60.groupby(['mes'], as_index = False).agg({'idade': 'count'})

In [129]:
list_meses = {'mes':[1,2,3,4,5,6,7,8,9,10,11,12]}

In [130]:
df_meses = pd.DataFrame(list_meses)

In [131]:
# pd.merge(df_meses,df_16_29,df_30_39,df_40_49,df_50_59,df_acima_60, on = 'mes', how = 'left') 

In [132]:
df_16_29 = df_16_29.T
df_30_39 = df_30_39.T
df_40_49 = df_40_49.T
df_50_59 = df_50_59.T
df_acima_60 = df_acima_60.T

In [133]:
df_aa = pd.concat([df_16_29,df_30_39,df_40_49,df_50_59,df_acima_60])

In [134]:
df_aa

,0,1,2,3,4,5
mes,5,6,7,8,9.0,10.0
idade,80,76,104,94,88.0,92.0
mes,5,6,7,8,9.0,10.0
idade,61,69,76,87,48.0,80.0
mes,5,6,7,8,9.0,10.0
idade,8,14,16,16,14.0,20.0
mes,5,6,7,8,9.0,10.0
idade,2,1,3,1,2.0,6.0
mes,6,7,9,10,NaN,NaN
idade,1,1,1,1,NaN,NaN


### Renomeando as colunas

In [135]:
df_16_29.rename(columns = {"idade":"16 A 29 ANOS"}, inplace = True)
df_30_39.rename(columns = {"idade":"30 A 39 ANOS"}, inplace = True)
df_40_49.rename(columns = {"idade":"40 A 49 ANOS"}, inplace = True)
df_50_59.rename(columns = {"idade":"50 A 59 ANOS"}, inplace = True)
df_acima_60.rename(columns = {"idade":"ACIMA DE 60 ANOS"}, inplace = True)

In [136]:
df_50_59

,0,1,2,3,4,5
mes,5,6,7,8,9,10
idade,2,1,3,1,2,6


In [137]:
df_16_29 = df_16_29.T
df_30_39 = df_30_39.T
df_40_49 = df_40_49.T
df_50_59 = df_50_59.T
df_acima_60 = df_acima_60.T

# SALVANDOS OS DATA FRAMES

In [138]:
writer = pd.ExcelWriter('Relatório de Atestados.xlsx', engine ='xlsxwriter')

### relatorio geral da quantidade de dias perdidos e da quantidade de licenças médicas gerais

df_atestados_meses.to_excel (writer, sheet_name = "relatorio_atestados", index = True, startrow = 1, startcol = 0)

workbook = writer.book

In [139]:
df_atestados_meses.to_excel (writer, sheet_name = "relatorio_atestados", index = True, startrow = 1, startcol = 0)

In [140]:
tam_linhas_atesados_qtd = df_atestados_meses.shape[0]+4

In [141]:
df_qtd_atestados_unidade.to_excel (writer, sheet_name = "relatorio_atestados", index = False, startrow = tam_linhas_atesados_qtd, startcol = 0)

In [142]:
tam_linhas_dias_atestados = df_qtd_atestados_unidade.shape[0]+15

In [143]:
df_qtd_dias_perdidos_unidade.to_excel (writer, sheet_name = "relatorio_atestados", index = False, startrow = tam_linhas_dias_atestados, startcol = 0)

In [144]:
df_final_genero.to_excel(writer, sheet_name = "relatorio_atestados", index = True, startrow = 115, startcol = 0)

In [145]:
writer.save()